In [ ]:
#include <opencv2/calib3d.hpp>
import numpy as np
import random as rd
import cv2

# Définition des paramètres intrasèques de la caméra
  # focale
fx = 60
fy = 40
  # centre
cx = 0
cy = 0

A = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]]) # matrice intrasèque de la caméra

# Génération de 3 angles aléatoire pour définir la rotation
rax = rd.uniform(0,2*np.pi)
pitch = rd.uniform(0,2*np.pi)
yaw = rd.uniform(0,2*np.pi)

Rx = np.array([[1,0,0],[0, np.cos(rax), -np.sin(rax)],[0, np.sin(rax), np.cos(rax)]])
Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)],[0,1,0],[-np.sin(pitch),0, np.cos(pitch)]])
Rz = np.array([[np.cos(yaw), -np.sin(yaw),0],[np.sin(yaw),np.cos(yaw),0],[0,0,1]])
R = Rz @ Ry @ Rx  # matrice de rotation


# Génération de la translation de la caméra

tx = rd.uniform(-500,500)
ty = rd.uniform(-500,500)
tz = rd.uniform(-500,500)
T = np.array([[tx], [ty], [tz]])

# Projection d'un point 3D vers 2D
def projection3D2D(point3D,T,R,A) :
  # point 3D = [ Xw, Yw, Zw ]
  # T : matrice de translation de la caméra : (3*1)
  # R : matrice de rotation : (3*3)

  PI = np.concatenate((np.eye(3),np.zeros(3,1)))  # (3*4)
  Rt = np.concatenate((R,T),axis=1)               # (3*4)
  Rt = np.concatenate((Rt,np.array([0,0,0,1])))   # (4*4)
  point3D_bis = np.concatenate((point3D,np.array([1])))   #(4*1)

  point2D = A @ PI @ Rt @ point3D_bis   # point 2D = [u, v, 1] (3*1)

  return point2D

# Création des couples de point 2D,3D
def point3Daleatoire() :
  return np.array([rd.uniform(-500,500),rd.uniform(-500,500),rd.uniform(-500,500)])

# points 3D
P1 = point3Daleatoire()     # (3*1)
P2 = point3Daleatoire()
P3 = point3Daleatoire()
P4 = point3Daleatoire()

points3D = np.concatenate((P1,P2,P3));     # (3*3)

# point 2D
p1 = projection3D2D(P1,T,R,fx,fy,cx,cy)   # (3*1)
p2 = projection3D2D(P2,T,R,fx,fy,cx,cy)
p3 = projection3D2D(P3,T,R,fx,fy,cx,cy)
p4 = projection3D2D(P4,T,R,fx,fy,cx,cy)

points2D = np.concatenate((p1,p2,p3));    # (3*3)

# Application de solveP3P pour retrouver la position de la caméra
retval, rvec, tvecs =  cv2.solveP3P(points3D,points2D,A,None)


# Mesure de l'erreur d'estimation
def erreur_estimation(R,T,R_P3P,T_P3P):
  erreur = 0
  for i in range(np.shape(R,1)) :
    for j in range(np.shape(R,2)):
      erreur += (R[i,j] - R_P3P[i,j])**2
  for i in range(np.shape(T,1)):
    erreur += (T[i] - T_P3P[i])**2
  return np.sqrt(erreur)




